In [0]:
%sql
-- Verify Silver tables are Delta
DESCRIBE DETAIL silver_sales_transactions;
DESCRIBE DETAIL silver_product_master;
DESCRIBE DETAIL silver_store_region;


format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,010c21a6-b2d7-4b80-a20c-4171b0811642,workspace.default.silver_store_region,null,,2025-12-23T00:13:12.392Z,2025-12-23T00:13:14.000Z,List(),List(),1,3458,"Map(delta.parquet.compression.codec -> zstd, delta.enableDeletionVectors -> true)",3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


In [0]:
from pyspark.sql.functions import col, when, current_timestamp, lit
from pyspark.sql.window import Window
from pyspark.sql.types import DecimalType, IntegerType
silver_sales_df = bronze_sales_df \
    .withColumn("quantity", col("quantity").cast(IntegerType())) \
    .withColumn("unit_price", col("unit_price").cast(DecimalType(10,2))) \
    .withColumn("discount", col("discount").cast(DecimalType(10,2))) \
    .withColumn(
        "correct_total_amount",
        (col("quantity") * col("unit_price")) - when(col("discount").isNull(), lit(0)).otherwise(col("discount"))
    ) \
    .withColumn("last_updated_ts", current_timestamp())
dedup_window = Window.partitionBy("transaction_id") \
                     .orderBy(col("last_updated_ts").desc())

silver_sales_dedup_df = silver_sales_df \
    .withColumn("rn", row_number().over(dedup_window)) \
    .filter(col("rn") == 1) \
    .drop("rn")
silver_sales_valid_df = silver_sales_dedup_df \
    .filter(col("quantity") > 0) \
    .filter(col("unit_price") > 0) \
    .filter(col("store_id").isNotNull()) \
    .filter(col("product_id").isNotNull())
silver_sales_valid_df.select(
    "transaction_id",
    "transaction_date",
    "store_id",
    "product_id",
    "quantity",
    "unit_price",
    "discount",
    col("correct_total_amount").cast(DecimalType(12,2)).alias("total_amount"),
    "currency",
    "last_updated_ts"
).write.format("delta") \
 .mode("overwrite") \
 .option("overwriteSchema", "true") \
 .saveAsTable("workspace.default.silver_sales_transactions")


In [0]:
from delta.tables import DeltaTable

silver_delta = DeltaTable.forName(spark, "silver_sales_transactions")

silver_delta.alias("t").merge(
    silver_sales_valid_df.alias("s"),   
    "t.transaction_id = s.transaction_id"
).whenMatchedUpdate(
    set={
        "transaction_date": "s.transaction_date",
        "store_id": "s.store_id",
        "product_id": "s.product_id",
        "quantity": "s.quantity",
        "unit_price": "s.unit_price",
        "discount": "s.discount",
        "total_amount": "s.correct_total_amount",
        "currency": "s.currency",
        "last_updated_ts": "s.last_updated_ts"
    }
).whenNotMatchedInsert(
    values={
        "transaction_id": "s.transaction_id",
        "transaction_date": "s.transaction_date",
        "store_id": "s.store_id",
        "product_id": "s.product_id",
        "quantity": "s.quantity",
        "unit_price": "s.unit_price",
        "discount": "s.discount",
        "total_amount": "s.correct_total_amount",
        "currency": "s.currency",
        "last_updated_ts": "s.last_updated_ts"
    }
).execute()


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
from delta.tables import DeltaTable

silver_delta = DeltaTable.forName(spark, "workspace.default.silver_sales_transactions")
incremental_df = spark.read.table("workspace.default.sales_transactions")

# Cast types to match Silver
from pyspark.sql.types import DecimalType, IntegerType

incremental_df = incremental_df \
    .withColumn("quantity", col("quantity").cast(IntegerType())) \
    .withColumn("unit_price", col("unit_price").cast(DecimalType(10,2))) \
    .withColumn("discount", col("discount").cast(DecimalType(10,2))) \
    .withColumn("correct_total_amount", (col("quantity")*col("unit_price")) - col("discount")) \
    .withColumn("last_updated_ts", current_timestamp())


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

dedup_window = Window.partitionBy("transaction_id").orderBy(col("last_updated_ts").desc())

incremental_df = incremental_df.withColumn("rn", row_number().over(dedup_window)) \
                               .filter(col("rn")==1).drop("rn")


In [0]:
silver_delta.alias("t").merge(
    incremental_df.alias("s"),
    "t.transaction_id = s.transaction_id"
).whenMatchedUpdate(
    set={
        "transaction_date": "s.transaction_date",
        "store_id": "s.store_id",
        "product_id": "s.product_id",
        "quantity": "s.quantity",
        "unit_price": "s.unit_price",
        "discount": "s.discount",
        "total_amount": "s.correct_total_amount",
        "currency": "s.currency",
        "last_updated_ts": "s.last_updated_ts"
    }
).whenNotMatchedInsert(
    values={
        "transaction_id": "s.transaction_id",
        "transaction_date": "s.transaction_date",
        "store_id": "s.store_id",
        "product_id": "s.product_id",
        "quantity": "s.quantity",
        "unit_price": "s.unit_price",
        "discount": "s.discount",
        "total_amount": "s.correct_total_amount",
        "currency": "s.currency",
        "last_updated_ts": "s.last_updated_ts"
    }
).execute()


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
-- Delta Time Travel – Query Previous Version
SELECT * 
FROM workspace.default.silver_sales_transactions VERSION AS OF 0;


product_id,store_id,transaction_id,transaction_date,quantity,unit_price,discount,total_amount,currency,last_updated_ts,ingestion_timestamp,source_system,ingestion_date,correct_total_amount,valid_store,valid_product,valid_quantity,valid_unit_price
1,101,TXN1,2025-12-13,3.0,800,0.0,2400,INR,2025-12-13T08:14:17.293Z,2025-12-23T00:07:43.306Z,raw_ingestion,2025-12-23,2400.0,true,true,true,true
3,401,TXN10,2025-12-21,2.0,120,5.0,235,EUR,2025-12-21T10:01:17.293Z,2025-12-23T00:07:43.306Z,raw_ingestion,2025-12-23,235.0,true,true,true,true
3,202,TXN100,2025-11-25,3.0,120,5.0,355,USD,2025-11-25T23:28:17.293Z,2025-12-23T00:07:43.306Z,raw_ingestion,2025-12-23,355.0,true,true,true,true
4,201,TXN101,2025-12-05,1.0,25,0.0,25,USD,2025-12-05T08:21:17.293Z,2025-12-23T00:07:43.306Z,raw_ingestion,2025-12-23,25.0,true,true,true,true
4,102,TXN104,2025-11-27,3.0,25,10.0,104,INR,2025-11-27T10:39:17.293Z,2025-12-23T00:07:43.306Z,raw_ingestion,2025-12-23,65.0,true,true,true,true
3,202,TXN105,2025-11-25,3.0,120,5.0,355,USD,2025-11-25T18:23:17.293Z,2025-12-23T00:07:43.306Z,raw_ingestion,2025-12-23,355.0,true,true,true,true
1,202,TXN106,2025-12-20,2.0,800,5.0,1595,USD,2025-12-21T03:44:17.293Z,2025-12-23T00:07:43.306Z,raw_ingestion,2025-12-23,1595.0,true,true,true,true
1,201,TXN107,2025-12-05,1.0,800,10.0,790,USD,2025-12-05T13:07:17.293Z,2025-12-23T00:07:43.306Z,raw_ingestion,2025-12-23,790.0,true,true,true,true
3,501,TXN108,2025-12-12,2.0,120,10.0,230,AUD,2025-12-13T04:30:17.293Z,2025-12-23T00:07:43.306Z,raw_ingestion,2025-12-23,230.0,true,true,true,true
1,102,TXN109,2025-12-19,1.0,800,5.0,821,INR,2025-12-19T15:20:17.293Z,2025-12-23T00:07:43.306Z,raw_ingestion,2025-12-23,795.0,true,true,true,true


In [0]:
%sql
-- Rollback Example
RESTORE TABLE workspace.default.silver_sales_transactions TO VERSION AS OF 0;


table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
13362,1,1,1,7460,13362


In [0]:
%sql
-- Optimize & ZORDER
OPTIMIZE workspace.default.silver_sales_transactions ZORDER BY (transaction_date, store_id);


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 13362), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1766449268370, 1766449271205, 8, 0, null, List(0, 0), null, 18, 18, 0, 0, null)"
